# Visualization
The following notebook includes everything necessary to display the Kalah Game visually.

<p style="background-color:yellow">NOTE: If you want to use this elsewhere you will need the images in the images folder of this directory</p>

In [ ]:
from ipycanvas import MultiCanvas,hold_canvas
from ipywidgets import Image
import math
import random
import IPython.display

NameError: name 'HTML' is not defined

### function: _state_to_dict
The function `_state_to_dict` is used to translate the current gamestate into a dictionary, that holds the house as key and the corresponding amount of seeds as value.

This is needed to be able to use the dictionary housemap which correlates the houses to their pixel positions.

In [ ]:
def _state_to_dict(state):
        dictionary = {}
        order = [['A','B','C','D','E','F','O'],['a','b','c','d','e','f','o']]
        for player in [0,1]:
            for house in range(7):
                dictionary[order[player][house]] = state[player][house]
        return dictionary

### Function: _draw_seeds
The function `_draw_seeds` reads the `state` of the game` and accordingly draws seeds on the respective houses. Each seed gets drawn in a random position inside the house with an additional random rotation.

To place the seeds accurately we must consider the width and height of the image of one seed. To do so we subtract the width and height from the final position chosen.

In [ ]:
def _draw_seeds(gamestate,canvas,housemap):
    HOUSE_RAD = 23
    HOUSE_OFFSETS = (90,125)
    seedsprites = [Image.from_file(f'images/Samen{i}.png') for i in range(1,6)]
    seed_width = 15
    seed_height = 15

    for house in gamestate:
        for seed in range(gamestate[house]):
            seed_sprite = seedsprites[random.randrange(5)]
            
            if house == 'O':
                centerX, centerY = (housemap['F'][0]-HOUSE_OFFSETS[0],random.randrange(housemap['F'][1],housemap['F'][1]+HOUSE_OFFSETS[1]))
            elif house == 'o':
                centerX, centerY = (housemap['F'][0]+HOUSE_OFFSETS[0]*6,random.randrange(housemap['F'][1],housemap['F'][1]+HOUSE_OFFSETS[1]))
            else:
                centerX, centerY = housemap[house]

            r = HOUSE_RAD * math.sqrt(random.random())
            theta = random.random() * 2 * math.pi
            x = centerX + r * math.cos(theta)
            y = centerY + r * math.sin(theta)
            canvas.save()

            #Rotate by random amount for variation
            canvas.translate(x, y)
            canvas.rotate(random.uniform(0.,math.pi))
            canvas.translate(-x, -y)
            canvas.draw_image(seed_sprite, x-seed_width, y-seed_height)
            canvas.restore()

### Function: _draw_numbers
The function `_draw_numbers` reads the `state` of the game and accordingly draws numbers that represent the amount of seeds on the respective houses. 

In [ ]:
def _draw_numbers(gamestate,canvas,housemap,turn):
    NUMBER_OFFSET_Y_BOT = 50
    NUMBER_OFFSET_Y_TOP = 60
    HOUSE_OFFSETS = (90,145)
    housenumbermap = {
        'F': (housemap['F'][0],housemap['F'][1]+NUMBER_OFFSET_Y_TOP),
        'E': (housemap['E'][0],housemap['E'][1]+NUMBER_OFFSET_Y_TOP),
        'D': (housemap['D'][0],housemap['D'][1]+NUMBER_OFFSET_Y_TOP),
        'C': (housemap['C'][0],housemap['C'][1]+NUMBER_OFFSET_Y_TOP),
        'B': (housemap['B'][0],housemap['B'][1]+NUMBER_OFFSET_Y_TOP),
        'A': (housemap['A'][0],housemap['A'][1]+NUMBER_OFFSET_Y_TOP),
        'a': (housemap['a'][0],housemap['a'][1]-NUMBER_OFFSET_Y_BOT),
        'b': (housemap['b'][0],housemap['b'][1]-NUMBER_OFFSET_Y_BOT),
        'c': (housemap['c'][0],housemap['c'][1]-NUMBER_OFFSET_Y_BOT),
        'd': (housemap['d'][0],housemap['d'][1]-NUMBER_OFFSET_Y_BOT),
        'e': (housemap['e'][0],housemap['e'][1]-NUMBER_OFFSET_Y_BOT),
        'f': (housemap['f'][0],housemap['f'][1]-NUMBER_OFFSET_Y_BOT),
        'O': (housemap['F'][0]-HOUSE_OFFSETS[0],housemap['F'][1]+HOUSE_OFFSETS[1]+HOUSE_OFFSETS[1]/5),
        'o': (housemap['F'][0]+HOUSE_OFFSETS[0]*6,housemap['F'][1]+HOUSE_OFFSETS[1]+HOUSE_OFFSETS[1]/5),
        
    }
    TURN_POS = (housemap['A'][0]+HOUSE_OFFSETS[0],62)

    canvas.font = '26px serif'
    canvas.text_align    = 'center'

    for key in housenumbermap:
        canvas.fill_text(gamestate[key],housenumbermap[key][0],housenumbermap[key][1])

    canvas.font = '22px serif'
    canvas.fill_text('Turn '+str(turn),TURN_POS[0],TURN_POS[1]) # TODO

### Function: draw_board
The function `draw_board` sets the required positional values for the functions `_draw_seeds` and `_draw_numbers`.

It also translates the current state of the game to a dictionary so that the positional values can be related to the corresponding seed amount in the respective houses.

Additionally this function creates a MultiCanvas for the display of the board. The first layer is the background and the second layer is the foreground.

Afterwards this function calls the functions `_draw_seed` and `_draw_numbers`.

Finally the finished canvas gets displayed.

In [ ]:
def draw_board(state, turn='?'):
    TOP_ROW_POSITION = (175,130)
    HOUSE_OFFSETS = (90,145)
    HOUSE_MAP={
        'F':TOP_ROW_POSITION,
        'E':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0],TOP_ROW_POSITION[1]),
        'D':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*2,TOP_ROW_POSITION[1]),
        'C':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*3,TOP_ROW_POSITION[1]),
        'B':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*4,TOP_ROW_POSITION[1]),
        'A':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*5,TOP_ROW_POSITION[1]),
        'a':(TOP_ROW_POSITION[0],TOP_ROW_POSITION[1]+HOUSE_OFFSETS[1]),
        'b':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0],TOP_ROW_POSITION[1]+HOUSE_OFFSETS[1]),
        'c':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*2,TOP_ROW_POSITION[1]+HOUSE_OFFSETS[1]),
        'd':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*3,TOP_ROW_POSITION[1]+HOUSE_OFFSETS[1]),
        'e':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*4,TOP_ROW_POSITION[1]+HOUSE_OFFSETS[1]),
        'f':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*5,TOP_ROW_POSITION[1]+HOUSE_OFFSETS[1]),
    }
    
    canvas = MultiCanvas(2,width=800, height=400)
    canvas[0].draw_image(Image.from_file('images/Board.png'),0,0)
    gamestate = _state_to_dict(state)
    
    with hold_canvas(canvas):
        _draw_seeds(gamestate,canvas[1],HOUSE_MAP)
        _draw_numbers(gamestate,canvas[1],HOUSE_MAP,turn)
    display(canvas)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b91c3ea7-d814-439b-837a-72fdc90697b1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>